# Intersection Numbers: One-Stop Guide

This notebook validates the core API and demonstrates how to compute intersection numbers $\langle \tau_{d_1}\cdots\tau_{d_n}\rangle_g$ using the normalized recursion table.

**Contents**
1. Package import and cache
2. Quick API tests
3. Building the recursion table (`iterate`)
4. Converting between $X$ (intersection numbers) and $R_X$ (normalized)
5. Examples: genus 0 and genus 1
6. How to query arbitrary tuples $(g,n,\alpha)$

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
from pathlib import Path
try:
    import wp
except ModuleNotFoundError:
    sys.path.insert(0, str(Path.cwd() / "src"))
    import wp

## 1) Quick API tests

In [ ]:
from fractions import Fraction
from wp.compute       import degree, is_stable
from wp.normalization import rx_from_x, x_from_rx

# Stability and degree
assert degree(0, 3) == 0 and is_stable(0, 3)
assert degree(1, 1) == 1 and is_stable(1, 1)

# Normalization round-trip on (g=0, n=4)
g, n, alpha = 0, 4, (1, 0, 0, 0)
X      = Fraction(1, 1)
RX     = rx_from_x(g, n, alpha, X)
X_back = x_from_rx(g, n, alpha, RX)
assert X_back == X

print("Basic API checks passed.")


Basic API checks passed.


## 2) Build / extend the recursion table
The function `wp.iterate(nmax, cache_path, max_workers, by_degree=True)` fills a cache of normalized values $R_X(g,n,\alpha)$ for all stable pairs with $3g-3+n\le n_{\text{max}}$.
Re-running is incremental; previously computed entries are reused.

In [5]:
ROOT = Path.cwd().parent          # repo/
cache_path = ROOT / "data" / "wp_rx.pkl"
cache_path.parent.mkdir(parents=True, exist_ok=True)

rx = wp.iterate(dmax=4, cache_path=str(cache_path), max_workers=None)
len(rx)

197172

## 3) Compute intersection numbers
We represent a multi-index by `alpha = (d1, d2, ..., dn)` sorted in **non-increasing** order.
Given the normalized value `RX = rx[(g,n,alpha)]`, convert to the intersection number via `wp.x_from_rx(g,n,alpha,RX)`.

In [7]:
def intersection_number(g:int, n:int, alpha:tuple[int,...], rx_table:dict):
    alpha = tuple(sorted(alpha, reverse=True))
    RX = rx_table.get((g,n,alpha))
    if RX is None:
        raise KeyError(f'Entry not found for (g={g}, n={n}, alpha={alpha}). Run iterate with larger nmax.')
    return wp.x_from_rx(g,n,alpha,RX)

# Genus 0 sanity: <tau_1 tau_0^3> = 1
val_g0 = intersection_number(0,4,(1,0,0,0), rx)
print('g=0, n=4, <tau_1 tau_0^3> =', val_g0)

# Genus 1 sanity: <tau_1> = 1/24
val_g1 = intersection_number(1,1,(1,), rx)
print('g=1, n=1, <tau_1> =', val_g1)

g=0, n=4, <tau_1 tau_0^3> = 1
g=1, n=1, <tau_1> = 1/24


## 4) Query helper
Use the convenience function below to request arbitrary tuples $(g,n,\alpha)$.

In [13]:
def query(g:int, alpha:tuple[int,...]):
    n = len(alpha)
    print(f'Request: g={g}, n={n}, alpha={tuple(sorted(alpha, reverse=True))}')
    X = intersection_number(g, n, alpha, rx)
    print('Intersection number =', X)

query(0, (2,0,0,0,0))  # <tau_2 tau_0^4> = 1 (genus 0)
query(1, (0,))         # <tau_0> = 1/24 (via normalization seeds)

Request: g=0, n=5, alpha=(2, 0, 0, 0, 0)
Intersection number = 1
Request: g=1, n=1, alpha=(0,)
Intersection number = 1/24
